# Naive Bayes

## What?

Naive Bayes is one of the simpler supervised techniques (and sometimes is criticized for its naivete) but it has found a nice home in the domain of text processing and information retrieval.  We will see how the algorithm works, how to use it, and why it works so well for text data.

## Where/When?

1. n << p (# of features)
2. n somewhat small __or__ 
3. n quite large
4. streams of input data (online learning)
5. not bounded by memory (usually)
6. Multi-class

## Why?

1. Independence Assumption
2. Generative algorithm (and also independence assumption)
3. Very performant... it's just counting!
4. Learns online by processing one data point at a time
5. see above (it is online and does incremental processing) ^^
6. Each class has a different parameterization (through CPTs)

### Why Text?

Why has Naive Bayes come to be used someaht exclusively for text.  Well... text data happens to fit very nicely to the bullet points of __Where/When__ to use Naive Bayes.  Due to bag of words featurization of text, the input feature matrix is often very wide (~10,000 - 50,000 columns/dimensions) and can even be greater than the number of data samples (n << p).

Also, we often encounter in the text classification problem the need for multiple classes.  We have the text of articles/documents and would like to classify the article to its appropriate topic/categorization (e.g. sports, politics, arts, etc.).

## Naive Bayes Intuition
We are going to be calculating the "probability" that an article belongs to each class. It's important to note that this will not be a true probability, but it will enable us to rank the classes to figure out which is the most likely.

Here's Bayes Rule applied to our scenario. *x* here is a word, *c* is the class (article topic). *X* is the document text of a specific document. The *x<sub>i</sub>*'s are all the words in the given document.

![bayes](images/bayes_rule.png)

### Naivite of Naive Bayes
We make a BIG assumption here that the occurrences of each word are *independent*. If they weren't, we wouldn't be able to just multiply all the individual word probabilities together to get the whole document probability. Hence the "Naive" in *Naive Bayes*.

Take a moment to think of how absurd this is. A document that contains the word "soccer" is probably more likely to contain the word "ball" than an average document. We assume that this is not true! But it turns out that this assumption doesn't hurt us too much and we still have a lot of predictive power. But the probabilities that we end up with are not at all true probabilities. We can only use them to compare which of the classes is the most likely.

## The Algorithm

For every document, we calculate the probability that the document belongs to each class and chose the class with the highest probability.

To do this, we need to calculate two things:
* **Priors:** The probability that a generic document belongs to each class: *P(c)*
* **Conditional Probabilities:** The probability that each word appears in each class: *P(x|c)*

So how do we actually get all those probabilities? *It's just counting!* We count occurrences in our training set to get approximations of the probabilities.

### Setup

$$D = [(x^{(1)}, c_{1}), (x^{(2)}, c_{2}), \ldots , (x^{(n)}, c_{n})]$$

$$where \quad x^{(i)} \equiv (x_{1}^{(i)}, \ldots, x_{d}^{(i)}) \quad \in \mathbb{R}^{d}$$

$$and$$

$$c \in \{1, \ldots, m\}$$

### The Model

![](images/nb.png)

We will assume a family of $P_{\theta}$ such that:

$$P_{\theta}(x, c) = P_{\theta}(x | c)P_{\theta}(c)$$

And applying Bayes Rul we get:

$$P_{\theta}(C=c_{i}|x_{1}, x_{2}, ... , x_{n}) = \frac{P_{\theta}(x_{1}, x_{2}, ... , x_{n}|C=c_{i})P_{\theta}(C=c_{i})}{\sum_{j}P_{\theta}(x_{1}, x_{2}, ... , x_{n}| C=c_{j})P_{\theta}(C=c_{j})}$$

And since we are only interested in the most probable class, we can get rid of the denominator:

$$P_{\theta}(C=c_{i}|x_{1}, x_{2}, ... , x_{n}) \propto P_{\theta}(x_{1}, x_{2}, ... , x_{n}|C=c_{i})P_{\theta}(C=c_{i})$$

And due to the naive assumption of conditional independence we can simplify the right side to:

$$P_{\theta}(C=c_{i}|x_{1}, x_{2}, ... , x_{n}) \propto P_{\theta}(x_{1} | c) P_{\theta}(c) \ldots P_{\theta}(x_{d} | c) P_{\theta}(c)$$

### Priors
The priors are the likelihood of each class. Based on the distribution of classes in our training set, we can assign a probability to each class:

![priors](images/priors.png)

Take a very simple example where 3 classes: sports, news, arts. There are 3 sports articles, 4 politics articles and 1 arts articles. This is 8 articles total. Here are our priors:

![priors examples](images/priors_example.png)


### Conditional Probability Table
The first step is building the *Conditional Probability Table (CPT)*. We would like to get, for every word, the count of the number of times it appears in each class. We are calculating the probability that a random word chosen from an article of class *c* is word *x*.

![conditional probability](images/conditional_prob.png)

Again, let's take our example. Let's look at the word "ball". Here are the occurrences in each of the 8 documents. We also need the word count of the documents.

| Article    | Occurrences of "ball" | Total # of words |
| :--------- | --------------------: | ---------------: |
| Sports 1   |                     5 |              101 |
| Sports 2   |                     7 |               93 |
| Sports 3   |                     0 |              122 |
| Politics 1 |                     0 |               39 |
| Politics 2 |                     0 |               81 |
| Politics 3 |                     0 |              142 |
| Politics 4 |                     0 |               77 |
| Arts 1     |                     2 |              198 |

Here are the values in the CPT table for the word "ball". We will do these calculations for all the words that appeared in the training documents.

![cpt example](images/cpt_example.png)


### Maximum Likelihood Estimation
We need to pull this all together to use these calculations to make a prediction. Here, *X* is the content of an article and *x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>, ...* are the words that make up the article.

![mle](images/mle.png)

We assign the article that has the largest probability. Note that since we made our incredibly naive assumption, these "probabilities" will not add to 1.

In our example, if we had the very short article of `The Giants beat the Nationals`, we would do the following calculations:

![mle example](images/mle_example.png)

The first probability is the prior, and the remaining come from the Conditional Probability Table. We make the same calculation for each of the 3 classes and choose the class with the biggest probability.

### Zero Probabilities
If a word has never before appeared in a document of a certain class, the probability will be 0. Since we are multiplying the probability, the whole probability becomes 0! We basically lose all information.

In the above example, since "ball" never appears in a politics article in the training set, if it appeared in a new article, we would say that there was a 0% chance that that was a politics article. But that's too harsh!

How can we fix this?

The simplest option is to replace these zeros with a really small value, like 0.00000001.
The better option is *Laplace Smoothing*.

#### Laplace Smoothing
Laplace Smoothing is assuming that every word has been seen by every article class one extra time. One way of thinking of it is that we add a new document to every class which has every word in it exactly once. This will guarantee that no value will be 0. So we modify our calculations of the probabilities slightly.

Here's our new conditional probability with Laplace smoothing.

![conditional probability with smoothing](images/conditional_prob_smoothing.png)

The standard is to use 1 as the smoothing constant, but in theory we could assume every word appeared half a time or something like this. This gives us a *smoothing constant*, which is a parameter we can tune:

![conditional probability with smoothing](images/conditional_prob_smoothing_constant.png)


### Log Likelihood
These probability values are going to get *really small*. Theoretically, this is not an issue, but when a computer makes the computation, we run the risk of *numerical underflow*. To keep our values bigger, we take the log. Recall that this is true: `log(ab) = log(a) + log(b)`

So when we calculated the MLE, we actually calculate the *log maximum likelihood error*.

![log mle](images/log_mle.png)

Recall that if `a > b` `log(a) > log(b)` so we can still find the maximum of the log likelihoods.

### Summary of Naive Bayes Algorithm
* **Training**: Calculate the priors and Conditional Probability Table
* **Predict**: Calculate the MLE for the new article for each label and pick the max

## One Bayes, many models

1. Multinomial Bayes (what we covered above)
2. Bernoulli Bayes
3. Gaussian Bayes

## The fun of Bayes

In theory you can use any prior distribution and any distribution for each feature.  This makes it quite adaptable and allows a data scientist easily incorporate domain knowledge.

### Example: Spam Detection

It is often illuminating to look at where (and why) techniques have come from to better understand their motivation, strengths, and weaknesses.  Naive Bayes technically can be traced back to the formulation of Bayes Theorem, but it was first extensively studied as a supervised classifier in the 1960s.

It really came into its own however in the [late 1990s](http://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering#History) as email became increasingly popular.  And most famously it was [popularized](http://www.paulgraham.com/spam.html) as a technique to filter spam emails by Paul Graham (of Y Combinator fame).

But why was spam email the "[killer app](http://en.wikipedia.org/wiki/Killer_application)" for Naive Bayes?

### Constraints of Service-wide Spam detection

Let us think about the problem of spam... If you are Google, you have millions of users who receive emails daily.  But the pattern of email is quite particular as compared to other datasets: a user receives a single email at a time and before you (Google) pass the email to their inbox you need to determine at that immediate moment whether or not it is spam.  So why not another classifier?

To be a successful feature of an email client, spam detection needs to:

1. Have __low False Negatives__ (you don't want your boss's email to go in your spam folder)
2. Be fast to predict (to deliver emails in a timely manner)
3. Train fast on all of your data... or incrementally update (the stream of emails is not stopping anytime soon)

With these constraints in mind, what would happen with another model (such as kNN) in the use case of spam detection.